## DAT2000 Våren 2024
## Oppvarming til 29. Januar

Som sist, så bruker vi smilefjesordningen:

- [Smilefjestilsyn på serveringssteder](https://data.norge.no/datasets/288aa74c-e3d3-492e-9ede-e71503b3bfd9)
- [Kravpunkter for smilefjestilsyn på serveringssteder](https://data.norge.no/datasets/1bf96c45-f8ac-4f2d-bca1-4e5ab4e7c6c6)

Kan være interessant å lese om karakterskalaen for disse kontrollene:

> Karakterskala: 
> 0 = Ingen brudd på regelverket funnet. Stort smil.
> 1 = Mindre brudd på regelverket som ikke krever oppfølging. Stort smil. 
> 2 = Brudd på regelverket som krever oppfølging. Strekmunn. 
> 3 = Alvorlig brudd på regelverket. Sur munn.
> 4 = Ikke aktuelt - Virksomheten har ikke denne aktiviteten ved tilsynsobjektet. Påvirker ikke smilefjeskarakter. 
> 5 = Ikke vurdert - Mattilsynet har ikke vurdert kravpunktet ved dette tilsynet. Påvirker ikke smilefjeskarakter. Dersom det hadde blitt avdekket mistanke om vesentlige eller åpenbare avvik i forbindelse med inspeksjonen, ville kravpunktet blitt vurdert.

fra: https://data.norge.no/datasets/288aa74c-e3d3-492e-9ede-e71503b3bfd9 (hentet 21.01.2024)

Disse kan du laste ned her: 
- [tilsyn.csv](https://hotell.difi.no/download/mattilsynet/smilefjes/tilsyn?download)
- [kravpunkter.csv](https://hotell.difi.no/download/mattilsynet/smilefjes/kravpunkter)

Plasser de i mappen jan29.

Deretter kan du installere noen pakker vi trenger:

```bash
pip install polars hvplot
```

## Kort om Polars
Polars er en minnebasert database for dataanalyse, som lar deg manipulere og analysere store datasett i minne. APIet er basert på såkalte DataFrames - vi har allerede sett disse i Pandas. Pandas er mer populært, men Polars har vesentlige forbedringer sammenlignet med Pandas:

- Pandas lar deg gjennomføre ulike operasjoner på datasett, mens Polars bygger disse som en spørring på datasettet, og lager en optimalisert spørringsplan når spørringen gjennomføres. Umiddelbar eksekvering kaller vi _eager_, mens når vi samler instruksjonene til en plan så kaller vi det _lazy_ eksekvering. 
- Polars er implementert i Rust og paralleliserer eksekveringen, Pandas bruker kun én tråd. 

Polars bruker flere av optimaliseringene for kolonnebaserte databaser som vi diskuterte på forelesning, blant annet så bruker den SIMD.  

Her er user guide: https://docs.pola.rs/ 
Her er API-dokumentasjonen: https://docs.pola.rs/py-polars/html/reference/ 

In [19]:
import polars as pl

In [20]:
df_t = pl.read_csv("tilsyn.csv", separator=";", dtypes={"postnr":str})
df_t

tilsynsobjektid,orgnummer,navn,adrlinje1,adrlinje2,postnr,poststed,tilsynid,sakref,status,dato,total_karakter,tilsynsbesoektype,tema1_no,tema1_nn,karakter1,tema2_no,tema2_nn,karakter2,tema3_no,tema3_nn,karakter3,tema4_no,tema4_nn,karakter4
str,i64,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,i64,str,str,i64,str,str,i64,str,str,i64
"""Z1501071230082…",914541034,"""Bakeriet i Bru…","""VANGSVEGEN 62""",null,"""2317""","""HAMAR""","""Z1601211510285…","""2016/6663""",0,21012016,1,0,"""Rutiner og led…","""Rutiner og lei…",1,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",0
"""Z1006120609250…",987770740,"""Aktiven skihei…","""Kvamskogen""",null,"""5600""","""NORHEIMSUND""","""Z1603101430592…","""2016/53185""",0,10032016,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1
"""Z1107281100559…",998066662,"""Golfkafeen""","""Belsjøveien 50…",null,"""1445""","""DRØBAK""","""Z1605111530190…","""2016/106865""",0,11052016,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1
"""Z1205111414434…",972297925,"""Fargeriet Café…","""Ivar Aasgaards…","""Arna Næringspa…","""5265""","""YTRE ARNA""","""Z1605201329276…","""2016/113661""",0,19052016,1,0,"""Rutiner og led…","""Rutiner og lei…",1,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0
"""Z1305061320139…",911860805,"""Marianne Baker…","""ODDEN 2""",null,"""5745""","""AURLAND""","""Z1601281551398…","""2016/20626""",0,28012016,3,0,"""Rutiner og led…","""Rutiner og lei…",2,"""Lokaler og uts…","""Lokaler og uts…",3,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",2
"""Z1301040959396…",975039129,"""Solbrød Nesttu…","""Østre Nesttunv…",null,"""5221""","""NESTTUN""","""Z1602011352244…","""2016/22950""",0,29012016,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1
"""Z1509021733238…",916327463,"""Suvi""","""Grønnegata 48""",null,"""9008""","""TROMSØ""","""Z1602181546569…","""2016/36480""",0,17022016,2,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",2,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0
"""Z1411211039297…",914430925,"""Cafe Jeanette""","""BRUGATA 16""",null,"""2380""","""BRUMUNDDAL""","""Z1602091334400…","""2016/29434""",0,9022016,0,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0
"""Z1006060857546…",979931069,"""Jafs Sem""","""Semsbyveien 94…",null,"""3170""","""SEM""","""Z1603301156458…","""2016/68720""",0,30032016,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0


In [21]:
# Vi kan skrive til CSV
df_t.write_csv("tilsyn2.csv")

In [22]:
# Vi kan skrive til Parquet
df_t.write_parquet("tilsyn.parquet")

In [23]:
#Vi kan velge en serie:
df_t["navn"]

navn
str
"""Bakeriet i Bru…"
"""Aktiven skihei…"
"""Golfkafeen"""
"""Fargeriet Café…"
"""Marianne Baker…"
"""Solbrød Nesttu…"
"""Suvi"""
"""Cafe Jeanette"""
"""Jafs Sem"""


In [24]:
# Vi kan velge én eller flere kolonner - dette blir en ny DataFrame
df_t[["navn", "postnr"]]

navn,postnr
str,str
"""Bakeriet i Bru…","""2317"""
"""Aktiven skihei…","""5600"""
"""Golfkafeen""","""1445"""
"""Fargeriet Café…","""5265"""
"""Marianne Baker…","""5745"""
"""Solbrød Nesttu…","""5221"""
"""Suvi""","""9008"""
"""Cafe Jeanette""","""2380"""
"""Jafs Sem""","""3170"""


In [25]:
# Men helst så burde vi bruke denne syntaksen til å velge flere kolonner:
df_t.select("navn", "postnr")

navn,postnr
str,str
"""Bakeriet i Bru…","""2317"""
"""Aktiven skihei…","""5600"""
"""Golfkafeen""","""1445"""
"""Fargeriet Café…","""5265"""
"""Marianne Baker…","""5745"""
"""Solbrød Nesttu…","""5221"""
"""Suvi""","""9008"""
"""Cafe Jeanette""","""2380"""
"""Jafs Sem""","""3170"""


In [26]:
# Vi kan også velge noe som er et uttrykk - legg merke til at "A" repeteres
# pl.lit("A") betyr en literal
df_t.select(pl.lit("A"), "postnr")

literal,postnr
str,str
"""A""","""2317"""
"""A""","""5600"""
"""A""","""1445"""
"""A""","""5265"""
"""A""","""5745"""
"""A""","""5221"""
"""A""","""9008"""
"""A""","""2380"""
"""A""","""3170"""


In [27]:
# Strengt tatt så kan vi bruke pl.col("postnr") for å si at vi vil ha kolonnen postnummer
df_t.select(pl.lit("A"), pl.col("postnr"))

literal,postnr
str,str
"""A""","""2317"""
"""A""","""5600"""
"""A""","""1445"""
"""A""","""5265"""
"""A""","""5745"""
"""A""","""5221"""
"""A""","""9008"""
"""A""","""2380"""
"""A""","""3170"""


In [28]:
# Vi kan filtrere på uttrykk:
df_t.filter(pl.col("postnr") == pl.lit("3179"))

tilsynsobjektid,orgnummer,navn,adrlinje1,adrlinje2,postnr,poststed,tilsynid,sakref,status,dato,total_karakter,tilsynsbesoektype,tema1_no,tema1_nn,karakter1,tema2_no,tema2_nn,karakter2,tema3_no,tema3_nn,karakter3,tema4_no,tema4_nn,karakter4
str,i64,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,i64,str,str,i64,str,str,i64,str,str,i64
"""Z2107141229368…",973740008,"""Grand Hotel Ås…","""HAVNEGATA 6""",null,"""3179""","""ÅSGÅRDSTRAND""","""Z2109141317346…","""2021/164977""",0,14092021,0,1,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1
"""Z2107141229368…",973740008,"""Grand Brygge""","""HAVNEGATA 6""",null,"""3179""","""ÅSGÅRDSTRAND""","""Z2307051443488…","""2023/144299""",0,5072023,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",0
"""Z2107141229368…",973740008,"""Grand Hotel Ås…","""HAVNEGATA 6""",null,"""3179""","""ÅSGÅRDSTRAND""","""Z2108171336262…","""2021/164977""",0,17082021,2,0,"""Rutiner og led…","""Rutiner og lei…",2,"""Lokaler og uts…","""Lokaler og uts…",2,"""Mat-håndtering…","""Mat-handtering…",2,"""Merking og spo…","""Merking og spo…",1
"""Z2003241304289…",924821124,"""Rådhuset Tratt…","""N C Nielsens g…",null,"""3179""","""ÅSGÅRDSTRAND""","""Z2307051308257…","""2023/144300""",0,5072023,0,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0
"""Z2107141229368…",973740008,"""Grand Hotel Ås…","""HAVNEGATA 6""",null,"""3179""","""ÅSGÅRDSTRAND""","""Z2311211243301…","""2023/249997""",0,21112023,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1
"""Z2107141229368…",973740008,"""Grand Hotel Ås…","""HAVNEGATA 6""",null,"""3179""","""ÅSGÅRDSTRAND""","""Z2206301352319…","""2022/136430""",0,30062022,2,0,"""Rutiner og led…","""Rutiner og lei…",2,"""Lokaler og uts…","""Lokaler og uts…",2,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",1
"""Z2107141229368…",973740008,"""Grand Hotel Ås…","""HAVNEGATA 6""",null,"""3179""","""ÅSGÅRDSTRAND""","""Z2207071441391…","""2022/136430""",0,7072022,0,1,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",1


In [29]:
# Dette endrer ikke df_t
df_t

tilsynsobjektid,orgnummer,navn,adrlinje1,adrlinje2,postnr,poststed,tilsynid,sakref,status,dato,total_karakter,tilsynsbesoektype,tema1_no,tema1_nn,karakter1,tema2_no,tema2_nn,karakter2,tema3_no,tema3_nn,karakter3,tema4_no,tema4_nn,karakter4
str,i64,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,i64,str,str,i64,str,str,i64,str,str,i64
"""Z1501071230082…",914541034,"""Bakeriet i Bru…","""VANGSVEGEN 62""",null,"""2317""","""HAMAR""","""Z1601211510285…","""2016/6663""",0,21012016,1,0,"""Rutiner og led…","""Rutiner og lei…",1,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",0
"""Z1006120609250…",987770740,"""Aktiven skihei…","""Kvamskogen""",null,"""5600""","""NORHEIMSUND""","""Z1603101430592…","""2016/53185""",0,10032016,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1
"""Z1107281100559…",998066662,"""Golfkafeen""","""Belsjøveien 50…",null,"""1445""","""DRØBAK""","""Z1605111530190…","""2016/106865""",0,11052016,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1
"""Z1205111414434…",972297925,"""Fargeriet Café…","""Ivar Aasgaards…","""Arna Næringspa…","""5265""","""YTRE ARNA""","""Z1605201329276…","""2016/113661""",0,19052016,1,0,"""Rutiner og led…","""Rutiner og lei…",1,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0
"""Z1305061320139…",911860805,"""Marianne Baker…","""ODDEN 2""",null,"""5745""","""AURLAND""","""Z1601281551398…","""2016/20626""",0,28012016,3,0,"""Rutiner og led…","""Rutiner og lei…",2,"""Lokaler og uts…","""Lokaler og uts…",3,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",2
"""Z1301040959396…",975039129,"""Solbrød Nesttu…","""Østre Nesttunv…",null,"""5221""","""NESTTUN""","""Z1602011352244…","""2016/22950""",0,29012016,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1
"""Z1509021733238…",916327463,"""Suvi""","""Grønnegata 48""",null,"""9008""","""TROMSØ""","""Z1602181546569…","""2016/36480""",0,17022016,2,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",2,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0
"""Z1411211039297…",914430925,"""Cafe Jeanette""","""BRUGATA 16""",null,"""2380""","""BRUMUNDDAL""","""Z1602091334400…","""2016/29434""",0,9022016,0,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0
"""Z1006060857546…",979931069,"""Jafs Sem""","""Semsbyveien 94…",null,"""3170""","""SEM""","""Z1603301156458…","""2016/68720""",0,30032016,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0


In [30]:
# Vi må gjøre assignment for å endre noe.
# Bruk ~ for negasjon, | for or og & for and. 
# Husk å sette parantes rundt uttrykkene.. 

df_t2 = df_t.filter((pl.col("postnr").eq(pl.lit("3179"))).or_(pl.col("poststed").eq(pl.lit("HORTEN"))))
df_t2

tilsynsobjektid,orgnummer,navn,adrlinje1,adrlinje2,postnr,poststed,tilsynid,sakref,status,dato,total_karakter,tilsynsbesoektype,tema1_no,tema1_nn,karakter1,tema2_no,tema2_nn,karakter2,tema3_no,tema3_nn,karakter3,tema4_no,tema4_nn,karakter4
str,i64,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,i64,str,str,i64,str,str,i64,str,str,i64
"""Z1006060856246…",885464122,"""Peppes Pizza A…","""TOLLBUGATA 1""",null,"""3187""","""HORTEN""","""Z1611161003111…","""2016/242241""",0,15112016,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",0
"""Z1405211258149…",973247409,"""Baltzer restau…","""JERNBANEGATA 1…",null,"""3187""","""HORTEN""","""Z1810191624429…","""2018/234355""",0,19102018,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0
"""Z1006060856452…",977259789,"""Sjømilitære Sa…","""Øvre vei 1""",null,"""3183""","""HORTEN""","""Z1901081303030…","""2019/6932""",0,8012019,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0
"""Z1006060856123…",972204668,"""Golden Crown""","""STORGATA 17""",null,"""3183""","""HORTEN""","""Z1711291656263…","""2017/252528""",0,29112017,0,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0
"""Z2306231638335…",924824182,"""Det hvite hus""","""Eskebekkveien …",null,"""3188""","""HORTEN""","""Z2306231702486…","""2023/136655""",0,23062023,1,0,"""Rutiner og led…","""Rutiner og lei…",1,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1
"""Z1601121030009…",915175635,"""Horten Mikrobr…","""STORGATA 20""",null,"""3181""","""HORTEN""","""Z1710251502470…","""2017/218075""",0,25102017,0,1,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1
"""Z1401061128526…",998414067,"""Oscars""","""STORGATA 21""",null,"""3181""","""HORTEN""","""Z2203311703594…","""2022/72584""",0,31032022,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",0
"""Z1401061128526…",998414067,"""Oscars""","""STORGATA 21""",null,"""3181""","""HORTEN""","""Z1604221140377…","""2016/83298""",0,22042016,0,1,"""Rutiner og led…","""Rutiner og lei…",1,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",0
"""Z1601261225402…",915915701,"""Blues´n""","""STORGATA 23""",null,"""3181""","""HORTEN""","""Z1811260835152…","""2018/264491""",0,23112018,0,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0


In [31]:
# Vi kan tilordne til kolonner
df_t3 = df_t.with_columns(
    (pl.col("postnr") + pl.lit(" ") + pl.col("poststed")).alias("postnr poststed")
)
df_t3

tilsynsobjektid,orgnummer,navn,adrlinje1,adrlinje2,postnr,poststed,tilsynid,sakref,status,dato,total_karakter,tilsynsbesoektype,tema1_no,tema1_nn,karakter1,tema2_no,tema2_nn,karakter2,tema3_no,tema3_nn,karakter3,tema4_no,tema4_nn,karakter4,postnr poststed
str,i64,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,i64,str,str,i64,str,str,i64,str,str,i64,str
"""Z1501071230082…",914541034,"""Bakeriet i Bru…","""VANGSVEGEN 62""",null,"""2317""","""HAMAR""","""Z1601211510285…","""2016/6663""",0,21012016,1,0,"""Rutiner og led…","""Rutiner og lei…",1,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",0,"""2317 HAMAR"""
"""Z1006120609250…",987770740,"""Aktiven skihei…","""Kvamskogen""",null,"""5600""","""NORHEIMSUND""","""Z1603101430592…","""2016/53185""",0,10032016,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1,"""5600 NORHEIMSU…"
"""Z1107281100559…",998066662,"""Golfkafeen""","""Belsjøveien 50…",null,"""1445""","""DRØBAK""","""Z1605111530190…","""2016/106865""",0,11052016,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1,"""1445 DRØBAK"""
"""Z1205111414434…",972297925,"""Fargeriet Café…","""Ivar Aasgaards…","""Arna Næringspa…","""5265""","""YTRE ARNA""","""Z1605201329276…","""2016/113661""",0,19052016,1,0,"""Rutiner og led…","""Rutiner og lei…",1,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0,"""5265 YTRE ARNA…"
"""Z1305061320139…",911860805,"""Marianne Baker…","""ODDEN 2""",null,"""5745""","""AURLAND""","""Z1601281551398…","""2016/20626""",0,28012016,3,0,"""Rutiner og led…","""Rutiner og lei…",2,"""Lokaler og uts…","""Lokaler og uts…",3,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",2,"""5745 AURLAND"""
"""Z1301040959396…",975039129,"""Solbrød Nesttu…","""Østre Nesttunv…",null,"""5221""","""NESTTUN""","""Z1602011352244…","""2016/22950""",0,29012016,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1,"""5221 NESTTUN"""
"""Z1509021733238…",916327463,"""Suvi""","""Grønnegata 48""",null,"""9008""","""TROMSØ""","""Z1602181546569…","""2016/36480""",0,17022016,2,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",2,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0,"""9008 TROMSØ"""
"""Z1411211039297…",914430925,"""Cafe Jeanette""","""BRUGATA 16""",null,"""2380""","""BRUMUNDDAL""","""Z1602091334400…","""2016/29434""",0,9022016,0,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0,"""2380 BRUMUNDDA…"
"""Z1006060857546…",979931069,"""Jafs Sem""","""Semsbyveien 94…",null,"""3170""","""SEM""","""Z1603301156458…","""2016/68720""",0,30032016,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0,"""3170 SEM"""


In [32]:
# Vi fikser opp i datoen
df_t = df_t.with_columns(
    dato=pl.col("dato").cast(str).str.pad_start(8, "0").str.to_date(format="%d%m%Y")
)

In [37]:
df_t.with_columns(
    year=pl.col("dato").dt.year()
)

tilsynsobjektid,orgnummer,navn,adrlinje1,adrlinje2,postnr,poststed,tilsynid,sakref,status,dato,total_karakter,tilsynsbesoektype,tema1_no,tema1_nn,karakter1,tema2_no,tema2_nn,karakter2,tema3_no,tema3_nn,karakter3,tema4_no,tema4_nn,karakter4,year
str,i64,str,str,str,str,str,str,str,i64,date,i64,i64,str,str,i64,str,str,i64,str,str,i64,str,str,i64,i32
"""Z1501071230082…",914541034,"""Bakeriet i Bru…","""VANGSVEGEN 62""",null,"""2317""","""HAMAR""","""Z1601211510285…","""2016/6663""",0,2016-01-21,1,0,"""Rutiner og led…","""Rutiner og lei…",1,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",0,2016
"""Z1006120609250…",987770740,"""Aktiven skihei…","""Kvamskogen""",null,"""5600""","""NORHEIMSUND""","""Z1603101430592…","""2016/53185""",0,2016-03-10,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1,2016
"""Z1107281100559…",998066662,"""Golfkafeen""","""Belsjøveien 50…",null,"""1445""","""DRØBAK""","""Z1605111530190…","""2016/106865""",0,2016-05-11,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1,2016
"""Z1205111414434…",972297925,"""Fargeriet Café…","""Ivar Aasgaards…","""Arna Næringspa…","""5265""","""YTRE ARNA""","""Z1605201329276…","""2016/113661""",0,2016-05-19,1,0,"""Rutiner og led…","""Rutiner og lei…",1,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0,2016
"""Z1305061320139…",911860805,"""Marianne Baker…","""ODDEN 2""",null,"""5745""","""AURLAND""","""Z1601281551398…","""2016/20626""",0,2016-01-28,3,0,"""Rutiner og led…","""Rutiner og lei…",2,"""Lokaler og uts…","""Lokaler og uts…",3,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",2,2016
"""Z1301040959396…",975039129,"""Solbrød Nesttu…","""Østre Nesttunv…",null,"""5221""","""NESTTUN""","""Z1602011352244…","""2016/22950""",0,2016-01-29,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1,2016
"""Z1509021733238…",916327463,"""Suvi""","""Grønnegata 48""",null,"""9008""","""TROMSØ""","""Z1602181546569…","""2016/36480""",0,2016-02-17,2,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",2,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0,2016
"""Z1411211039297…",914430925,"""Cafe Jeanette""","""BRUGATA 16""",null,"""2380""","""BRUMUNDDAL""","""Z1602091334400…","""2016/29434""",0,2016-02-09,0,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0,2016
"""Z1006060857546…",979931069,"""Jafs Sem""","""Semsbyveien 94…",null,"""3170""","""SEM""","""Z1603301156458…","""2016/68720""",0,2016-03-30,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0,2016


In [38]:
# Vi kan lage mer kompliserte uttrykk
# .str inneholder en del streng-metoder, viktig at datatypen er string i dette tilfellet.
df_t.with_columns(
    (pl.col("poststed").str.slice(0,1) + 
     pl.col("poststed").str.slice(1, None).str.to_lowercase()).alias("poststed"))

tilsynsobjektid,orgnummer,navn,adrlinje1,adrlinje2,postnr,poststed,tilsynid,sakref,status,dato,total_karakter,tilsynsbesoektype,tema1_no,tema1_nn,karakter1,tema2_no,tema2_nn,karakter2,tema3_no,tema3_nn,karakter3,tema4_no,tema4_nn,karakter4
str,i64,str,str,str,str,str,str,str,i64,date,i64,i64,str,str,i64,str,str,i64,str,str,i64,str,str,i64
"""Z1501071230082…",914541034,"""Bakeriet i Bru…","""VANGSVEGEN 62""",null,"""2317""","""Hamar""","""Z1601211510285…","""2016/6663""",0,2016-01-21,1,0,"""Rutiner og led…","""Rutiner og lei…",1,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",0
"""Z1006120609250…",987770740,"""Aktiven skihei…","""Kvamskogen""",null,"""5600""","""Norheimsund""","""Z1603101430592…","""2016/53185""",0,2016-03-10,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1
"""Z1107281100559…",998066662,"""Golfkafeen""","""Belsjøveien 50…",null,"""1445""","""Drøbak""","""Z1605111530190…","""2016/106865""",0,2016-05-11,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1
"""Z1205111414434…",972297925,"""Fargeriet Café…","""Ivar Aasgaards…","""Arna Næringspa…","""5265""","""Ytre arna""","""Z1605201329276…","""2016/113661""",0,2016-05-19,1,0,"""Rutiner og led…","""Rutiner og lei…",1,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0
"""Z1305061320139…",911860805,"""Marianne Baker…","""ODDEN 2""",null,"""5745""","""Aurland""","""Z1601281551398…","""2016/20626""",0,2016-01-28,3,0,"""Rutiner og led…","""Rutiner og lei…",2,"""Lokaler og uts…","""Lokaler og uts…",3,"""Mat-håndtering…","""Mat-handtering…",1,"""Merking og spo…","""Merking og spo…",2
"""Z1301040959396…",975039129,"""Solbrød Nesttu…","""Østre Nesttunv…",null,"""5221""","""Nesttun""","""Z1602011352244…","""2016/22950""",0,2016-01-29,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",1
"""Z1509021733238…",916327463,"""Suvi""","""Grønnegata 48""",null,"""9008""","""Tromsø""","""Z1602181546569…","""2016/36480""",0,2016-02-17,2,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",2,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0
"""Z1411211039297…",914430925,"""Cafe Jeanette""","""BRUGATA 16""",null,"""2380""","""Brumunddal""","""Z1602091334400…","""2016/29434""",0,2016-02-09,0,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",0,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0
"""Z1006060857546…",979931069,"""Jafs Sem""","""Semsbyveien 94…",null,"""3170""","""Sem""","""Z1603301156458…","""2016/68720""",0,2016-03-30,1,0,"""Rutiner og led…","""Rutiner og lei…",0,"""Lokaler og uts…","""Lokaler og uts…",1,"""Mat-håndtering…","""Mat-handtering…",0,"""Merking og spo…","""Merking og spo…",0


In [39]:
df_k = pl.read_csv("kravpunkter.csv", separator=";").with_columns(
    pl.col("dato").cast(str).str.pad_start(8, "0").str.to_date(format="%d%m%Y")
)
df_k

tilsynid,dato,ordningsverdi,kravpunktnavn_no,kravpunktnavn_nn,karakter,tekst_no,tekst_nn
str,date,f64,str,str,i64,str,str
"""Z1601041508412…",2016-01-04,1.1,"""Synlig rapport…","""Synleg rapport…",5,"""Ikke vurdert""","""Ikkje vurdert"""
"""Z1601041508412…",2016-01-04,1.2,"""Meldeplikt for…","""Meldeplikt for…",5,"""Ikke vurdert""","""Ikkje vurdert"""
"""Z1601041508412…",2016-01-04,1.3,"""Ansvaret til d…","""Ansvaret til d…",5,"""Ikke vurdert""","""Ikkje vurdert"""
"""Z1601041508412…",2016-01-04,1.4,"""Internkontroll…","""Internkontroll…",0,null,null
"""Z1601041508412…",2016-01-04,1.5,"""Farevurdering …","""Farevurdering …",5,"""Ikke vurdert""","""Ikkje vurdert"""
"""Z1601041508412…",2016-01-04,1.6,"""Opplæring og k…","""Opplæring og k…",5,"""Ikke vurdert""","""Ikkje vurdert"""
"""Z1601041508412…",2016-01-04,2.1,"""Generelt - pla…","""Generelt - pla…",2,"""Mangler ved hy…","""Manglar ved hy…"
"""Z1601041508412…",2016-01-04,2.2,"""Lokaler - spes…","""Lokaler - spes…",5,"""Ikke vurdert""","""Ikkje vurdert"""
"""Z1601041508412…",2016-01-04,2.3,"""Renhold""","""Reinhald""",0,null,null


In [40]:
# Vi kan joine:
df_j = df_t.select(
    pl.col("navn"), pl.col("orgnummer"), pl.col("tilsynid")
    ).join(
    df_k.select(
        pl.col("tilsynid"), pl.col("ordningsverdi"), pl.col("karakter"), pl.col("kravpunktnavn_no")
    ), on="tilsynid",how="left")
df_j

navn,orgnummer,tilsynid,ordningsverdi,karakter,kravpunktnavn_no
str,i64,str,f64,i64,str
"""Bakeriet i Bru…",914541034,"""Z1601211510285…",1.1,5,"""Synlig rapport…"
"""Bakeriet i Bru…",914541034,"""Z1601211510285…",1.2,5,"""Meldeplikt for…"
"""Bakeriet i Bru…",914541034,"""Z1601211510285…",1.3,5,"""Ansvaret til d…"
"""Bakeriet i Bru…",914541034,"""Z1601211510285…",1.4,1,"""Internkontroll…"
"""Bakeriet i Bru…",914541034,"""Z1601211510285…",1.5,5,"""Farevurdering …"
"""Bakeriet i Bru…",914541034,"""Z1601211510285…",1.6,5,"""Opplæring og k…"
"""Bakeriet i Bru…",914541034,"""Z1601211510285…",2.1,0,"""Generelt - pla…"
"""Bakeriet i Bru…",914541034,"""Z1601211510285…",2.2,0,"""Lokaler - spes…"
"""Bakeriet i Bru…",914541034,"""Z1601211510285…",2.3,1,"""Renhold"""


In [41]:
# Vi kan gruppere og aggregere
df_j.group_by("tilsynid", "orgnummer", "navn").agg(
    (pl.col("karakter") == 3).any().alias("noen_karakter_3")
)

tilsynid,orgnummer,navn,noen_karakter_3
str,i64,str,bool
"""Z1602091334400…",914430925,"""Cafe Jeanette""",false
"""Z1604261412189…",996694216,"""Klosterhagen H…",false
"""Z1609091521354…",974052393,"""Magnussen Kaff…",false
"""Z1609121109586…",896723782,"""Prestige Sushi…",false
"""Z1606100821185…",816392012,"""Pronto Pizza A…",false
"""Z1607181524507…",979205031,"""Evergrenn Haug…",false
"""Z1607270809080…",972404276,"""Sølenstua""",false
"""Z1609211252343…",982603552,""" Bajazzo Høyde…",false
"""Z1608101630469…",971945389,"""Nye Bamboo Gar…",false


In [42]:
# Vi kan kombinere flere uttrykk
# Vi kan gruppere og aggregere og deretter sortere
df_j.group_by("tilsynid", "orgnummer", "navn").agg(
    (pl.col("karakter") == 3).any().alias("noen_karakter_3")
).sort("noen_karakter_3", descending=True)

tilsynid,orgnummer,navn,noen_karakter_3
str,i64,str,bool
"""Z1601281551398…",911860805,"""Marianne Baker…",true
"""Z1602251117043…",971785551,"""Fregatten sush…",true
"""Z2110210951419…",915927068,"""Mae Kanya's Th…",true
"""Z2306071452174…",921187777,"""Oslo kebab & p…",true
"""Z1907241902120…",914291933,"""Estar Sushi""",true
"""Z1608291250214…",982466709,"""Paradis pizza""",true
"""Z2312291044459…",832514772,"""Hos Martin Gol…",true
"""Z1703141429549…",976442466,"""Jonas B. Gunde…",true
"""Z2202171616196…",979520999,"""Terrassen Kafe…",true


In [45]:
# Vi bruker .lazy() for å lage en spørringsplan, og .collect() for å eksekvere spørringsplanen. 
# Vi bruker .scan_csv() for å bare lage en plan om å lese CSV.
# Optimaliseringene gjør at vi kun leser inn de kolonnene som faktisk blir brukt!
# Dette kalles en LazyFrame, ikke en DataFrame
lf_t = pl.scan_csv("tilsyn.csv", separator=";", dtypes={"postnr":str})
df_t = lf_t.select("orgnummer", "navn", "dato", "total_karakter", "poststed").with_columns(
    pl.col("dato").cast(str).str.pad_start(8, "0").str.to_date(format="%d%m%Y")
).filter(
    pl.col("total_karakter").is_in([2,3])
).filter(
    pl.col("dato").dt.year() == 2018
).filter(
    pl.col("poststed") == "HORTEN"
).collect()
df_t

orgnummer,navn,dato,total_karakter,poststed
i64,str,date,i64,str
915175635,"""Horten Mikrobr…",2018-10-24,2,"""HORTEN"""
972876909,"""Rosalinda`s Or…",2018-10-18,2,"""HORTEN"""
